<a href="https://colab.research.google.com/github/IA2021UR/trabajo-ia-en-la-comunidad-de-la-rioja-equipo-3/blob/main/Inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Carga de los datasets

In [3]:
!wget https://www.dropbox.com/s/mvhvpxm1n2k02rv/stitching.zip?dl=1 -O stitching.zip

--2021-04-27 15:36:31--  https://www.dropbox.com/s/mvhvpxm1n2k02rv/stitching.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/mvhvpxm1n2k02rv/stitching.zip [following]
--2021-04-27 15:36:31--  https://www.dropbox.com/s/dl/mvhvpxm1n2k02rv/stitching.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc60cb3e49d4ff0635b83027595b.dl.dropboxusercontent.com/cd/0/get/BNaXFdvIJimUYsVhnGent_Gxti6tavByw6R7C0OBuNzTJUZnl0uAQRP8Lv6omIHmYaXcxCGnWWlWlRC93XpRoRA2oXxYYwW02A1NTNReYpWalSzK7UhKclMw5a2zWw6SmC0DC-0UgoUbNrn50wyx7uSN/file?dl=1# [following]
--2021-04-27 15:36:32--  https://uc60cb3e49d4ff0635b83027595b.dl.dropboxusercontent.com/cd/0/get/BNaXFdvIJimUYsVhnGent_Gxti6tavByw6R7C0OBuNzTJUZnl0uAQRP8Lv6omIHmYaXcxCGnWWlWlRC93XpRo

In [4]:
!unzip stitching.zip

Archive:  stitching.zip
   creating: Abalos/
  inflating: Abalos/Abalos_04.jpg    
  inflating: Abalos/Abalos_05.jpg    
  inflating: Abalos/Abalos_06.jpg    
  inflating: Abalos/Abalos_07.jpg    
  inflating: Abalos/Abalos_01.jpg    
  inflating: Abalos/Abalos_02.jpg    
  inflating: Abalos/Abalos_03.jpg    
   creating: Brieva/
  inflating: Brieva/Brieva-13.jpg    
  inflating: Brieva/Brieva-14.jpg    
  inflating: Brieva/Brieva-16.jpg    
  inflating: Brieva/Brieva-23.jpg    
  inflating: Brieva/Brieva-25.jpg    
  inflating: Brieva/Brieva-27.jpg    
  inflating: Brieva/Brieva-28 A.jpg  
  inflating: Brieva/Brieva-28 B.jpg  
  inflating: Brieva/Brieva-28 C.jpg  
  inflating: Brieva/Brieva-29.jpg    
  inflating: Brieva/Brieva-30.jpg    
  inflating: Brieva/Brieva-32.jpg    
  inflating: Brieva/Brieva-34 bis.jpg  
  inflating: Brieva/Brieva-34.jpg    
  inflating: Brieva/Brieva-35 A.jpg  
  inflating: Brieva/Brieva-35 B.jpg  
  inflating: Brieva/Brieva-35 C.jpg  
  inflating: Brieva/

In [1]:
 !pip install imutils